# 1. Money Markets

1.1

1. False
2. True

1.2

1. Yes SOFR and Treasury-bill rates are correlated conceptually. In money markets, both rates are considered to be the risk-free rates (alongside the FED funds rate). Additionally, both are short-term rates

2. No:
- SOFR is historic and overnight (based on previous night's repo transaction). T-bill rates are not as short-term, they can have a maximum maturity of 52 weeks
- Additionally, T-bills enter the market via dealers with their rates being determinted intially by an auction among the ~20 dealers, and enter the economy via the secondary market

1.3

Series with high serial correlation: SOFR rates, Fed Funds rates, and T-bill yields 

1.4

1. False
2. True
3. False

1.5

1. Yes. The overnight repo and T-bill will offer similar rates which will typically be higher than the interest rates offered by banks on savings
2. Yes. The overnight repo and T-bills are nearly risk free, while savings accounts for such large amounts is risky since there's a chance the bank collapses within the 5 days, and the FDIC insurance protection offered on the savings account will not suffice for the large sums held by asset managers

# 2. Treasury Debt

In [60]:
import pandas as pd

exam_three_data = 'exam_3_data.xlsx'

tabs = pd.ExcelFile(exam_three_data).sheet_names
tabs

['treasury specs', 'NVDA', 'adjusted prices']

In [61]:
df = pd.read_excel(exam_three_data, "treasury specs")
df

df = df.set_index('Unnamed: 0')
df = df.transpose()

df_no_tips = df.drop(index="CTII10 Govt") # Ignore TIPS note for now
df_no_tips

Unnamed: 0,name,issue date,maturity date,amount issued,ttm (years),inflation adjustment,coupon rate,accrued interest,price quoted (bid),ytm (bid)
CB3 Govt,TREASURY BILL,2024-10-03,2025-10-02,207649000000,0.238193,1,0,0,4.2575,4.362
CT10 Govt,US TREASURY N/B,2025-05-15,2035-05-15,96526000000,9.853525,1,0.0425,0.612092,99-06+,4.35


# 2.1

In [62]:
# 1. Calculate the bid price in dollars and quoted in decimals

# CB3 Govt (Treasury Bill) -> price quoted is discount
# Price =  Face Value × (1 - (Discount Rate × Days to Maturity / 360))
t_bill_discount_rate = df_no_tips.loc['CB3 Govt', 'price quoted (bid)'] / 100
t_bills_days_to_mat = round(df_no_tips.loc['CB3 Govt', 'ttm (years)'] * 365)
face_value = 100
t_bill_bid_price = face_value * (1 - (t_bill_discount_rate * t_bills_days_to_mat / 360))

# CT10 Govt (US Treasury Note) -> 99-06+: 99 => 99, 06 => 6/32, + => 1/64
t_note_bid_price = 99 + 6/32 + 1/64

df_no_tips['bid_price_decimal'] = [t_bill_bid_price, t_note_bid_price]
df_no_tips

Unnamed: 0,name,issue date,maturity date,amount issued,ttm (years),inflation adjustment,coupon rate,accrued interest,price quoted (bid),ytm (bid),bid_price_decimal
CB3 Govt,TREASURY BILL,2024-10-03,2025-10-02,207649000000,0.238193,1,0,0,4.2575,4.362,98.971104
CT10 Govt,US TREASURY N/B,2025-05-15,2035-05-15,96526000000,9.853525,1,0.0425,0.612092,99-06+,4.35,99.203125


# 2.2

In [63]:
# 2. Dirty Prices: Price treasury with accrued interest
df_no_tips['dirty_price'] = df_no_tips['bid_price_decimal'] + df_no_tips['accrued interest']
df_no_tips

Unnamed: 0,name,issue date,maturity date,amount issued,ttm (years),inflation adjustment,coupon rate,accrued interest,price quoted (bid),ytm (bid),bid_price_decimal,dirty_price
CB3 Govt,TREASURY BILL,2024-10-03,2025-10-02,207649000000,0.238193,1,0,0,4.2575,4.362,98.971104,98.971104
CT10 Govt,US TREASURY N/B,2025-05-15,2035-05-15,96526000000,9.853525,1,0.0425,0.612092,99-06+,4.35,99.203125,99.815217


# 2.3

In [64]:
from scipy.optimize import fsolve

def bond_price(n, c, ytm, F=100):
    price = 0
    
    for i in range(1, 2*n + 1):
       price += ((F*c/2)) / ((1 + ytm/2) ** i)
    
    price += (F) / ((1 + ytm/2) ** (2*n))
    
    return price

# bond_price(n=10, c=0.0425, ytm=4.35/100)

def ytm_solver(y, n, c, actual_bond_price):
   return bond_price(n, c, y) - actual_bond_price

n = 10
c = 0.0425
actual_bond_price = 99.203125 # clean price
    
starting_ytm_guess = 0.05 # can use any reasonable value. 5% because we know current treasuries have YTM btw 4.5 & 5.5

ytm_answer = fsolve(ytm_solver, starting_ytm_guess, args=(n, c, actual_bond_price))
ytm = ytm_answer[0]
computed_ytm_percentage = ytm * 100
computed_ytm_percentage

4.349118120706895

In [65]:
from scipy.optimize import fsolve

def bond_price(n, c, ytm, F=100):
    price = 0
    
    for i in range(1, 2*n + 1):
       price += ((F*c/2)) / ((1 + ytm/2) ** i)
    
    price += (F) / ((1 + ytm/2) ** (2*n))
    
    return price


def compute_ytm(n, c, actual_bond_price): # actual bond price -> clean price

    def ytm_solver(y, n, c, actual_bond_price):
       return bond_price(n, c, y) - actual_bond_price

    starting_ytm_guess = 0.05 # can use any reasonable value. 5% because we know current treasuries have YTM btw 4.5 & 5.5

    ytm_answer = fsolve(ytm_solver, starting_ytm_guess, args=(n, c, actual_bond_price))
    ytm = ytm_answer[0]
    computed_ytm_percentage = ytm * 100
    return computed_ytm_percentage

t_bill_computed_ytm = compute_ytm(n=0, c=0, actual_bond_price=98.971104)
t_note_computed_ytm = compute_ytm(n=10, c=0.0425, actual_bond_price=99.203125)

df_no_tips["computed_ytm"] = [t_bill_computed_ytm, t_note_computed_ytm]
df_no_tips

# Computed YTM for the T note is very close (4.349 vs 4.35), but the T bill's is not as close (5 vs 4.3)

C:\Users\DELL\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Unnamed: 0,name,issue date,maturity date,amount issued,ttm (years),inflation adjustment,coupon rate,accrued interest,price quoted (bid),ytm (bid),bid_price_decimal,dirty_price,computed_ytm
CB3 Govt,TREASURY BILL,2024-10-03,2025-10-02,207649000000,0.238193,1,0,0,4.2575,4.362,98.971104,98.971104,5.000000
CT10 Govt,US TREASURY N/B,2025-05-15,2035-05-15,96526000000,9.853525,1,0.0425,0.612092,99-06+,4.35,99.203125,99.815217,4.349118


# 2.4

In [66]:
# YTM of 10 year nominal jumps up 20bps => YTM of T-notes (CT10 Govt) up 0.20 % 

prev_price = df_no_tips.loc['CT10 Govt', 'bid_price_decimal']
new_price = bond_price(n=10, c=0.0425, ytm=(4.35+0.2)/100)

print(f"T-note: Old price = {prev_price}, updated price = {new_price}")
print("T-bill: price unaffected")

T-note: Old price = 99.203125, updated price = 97.61114895623012
T-bill: price unaffected


# 2.5

In [67]:
nominal_ytm = df.loc['CT10 Govt', 'ytm (bid)']
tips_ytm = df.loc['CTII10 Govt', 'ytm (bid)']

breakeven = nominal_ytm - tips_ytm

print(f"Over the next 10 years, market anticipates inflation of ~{breakeven}% per year")

Over the next 10 years, market anticipates inflation of ~2.3509999999999995% per year


# 2.6

In [68]:
tips_df = df[df["name"] == "TSY INFL IX N/B"].copy()

# Price: 101-03 => 101 + 3/32
tips_price = 101 + 3/32
tips_df['bid_price_decimal'] = tips_price

# YTM:
tips_computed_ytm = compute_ytm(n=10, c=0.02125, actual_bond_price=101.09375)
tips_df["computed_ytm"] = tips_computed_ytm
tips_df # Very accurate (2 vs 1.999)

Unnamed: 0,name,issue date,maturity date,amount issued,ttm (years),inflation adjustment,coupon rate,accrued interest,price quoted (bid),ytm (bid),bid_price_decimal,computed_ytm
CTII10 Govt,TSY INFL IX N/B,2025-01-31,2035-01-15,57625000000,9.524983,1.01691,0.02125,1.015539,101-03,1.999,101.09375,2.003756


# 2.7

In [69]:
tips_df

principal = df.loc['CTII10 Govt', 'amount issued']
inflation = df.loc['CTII10 Govt', 'inflation adjustment']
semi_annual_coupon = df.loc['CTII10 Govt', 'coupon rate'] / 2 # TIPS have Semi-annual payments, so use half the coupon rate

# Total dollars of coupon it will pay out at the next date:
next_coupon_payment = principal * inflation * semi_annual_coupon
print(f"Coupon Pay Out at the next rate = {next_coupon_payment}")

Coupon Pay Out at the next rate = 622619036.71875


# 3. Equity

# 3.1

In [70]:
df = pd.read_excel(exam_three_data, "NVDA")
df

,Date,Unadjusted Price
0,2024-06-06,1209.98
1,2024-06-07,1208.88
2,2024-06-10,121.79
3,2024-06-11,120.91


In [71]:
# Derive adjusted price accounting for stock split
df['adjusted_price'] = df['Unadjusted Price']
df.loc[0:1, 'adjusted_price'] = df['adjusted_price'] / 10

# add dividend col
df['Dividend'] = 0
df.loc[df['Date'] == "2024-06-10", 'Dividend'] = 0.1
df

# Compute daily (Dividend-Adjusted) return:
prev_val = df["adjusted_price"].shift(1)
todays_val = df["adjusted_price"] + df["Dividend"]
df["daily_return_percent"] = ((todays_val - prev_val) / prev_val )* 100
df

,Date,Unadjusted Price,adjusted_price,Dividend,daily_return_percent
0,2024-06-06,1209.98,120.998,0.0,NaN
1,2024-06-07,1208.88,120.888,0.0,-0.090911
2,2024-06-10,121.79,121.790,0.1,0.828866
3,2024-06-11,120.91,120.910,0.0,-0.722555


# 3.2

In [72]:
df = pd.read_excel(exam_three_data, "adjusted prices")

def get_returns_percent(df, col_name):
    df[f"{col_name}_returns_percent"] = df[col_name].pct_change() * 100
    return df

df = get_returns_percent(df, 'SPY')
df = get_returns_percent(df, 'NVDA')
df

,date,SPY,NVDA,SPY_returns_percent,NVDA_returns_percent
0,2020-06-03,293.9492,8.7395,NaN,NaN
1,2020-06-04,293.1771,8.7405,-0.262664,0.011442
2,2020-06-05,300.6911,8.8935,2.562956,1.750472
3,2020-06-08,304.3257,8.7789,1.208749,-1.288582
4,2020-06-09,302.0564,9.0192,-0.745681,2.737245
...,...,...,...,...,...
1010,2024-06-07,534.0100,120.8781,-0.121573,-0.090918
1011,2024-06-10,535.6600,121.7800,0.308983,0.746124
1012,2024-06-11,536.9500,120.9100,0.240824,-0.714403
1013,2024-06-12,541.3600,125.2000,0.821306,3.548094


In [73]:
import math

def annualized_stats(df, col_name):
    mean_returns = df[col_name].mean() * 252
    vol_returns = df[col_name].std() * math.sqrt(252)
        
    print(f"{col_name}: Annualized Mean Return: {mean_returns}, Annualized Volatility Return = {vol_returns} ")
    
annualized_stats(df, 'SPY_returns_percent')
annualized_stats(df, 'NVDA_returns_percent')

SPY_returns_percent: Annualized Mean Return: 16.70434073878169, Annualized Volatility Return = 17.153514180393294 
NVDA_returns_percent: Annualized Mean Return: 80.14424454347815, Annualized Volatility Return = 51.43549115322548 


# 3.3

In [74]:
import statsmodels.api as sm

# Independant variable (SPY_returns_percent) cannot contain inf or nan values. Drop rows with NaNs
df_clean = df.dropna()

X = df_clean['SPY_returns_percent']
Y = df_clean['NVDA_returns_percent']

X = sm.add_constant(X)
model = sm.OLS(df_clean['NVDA_returns_percent'], X)
results = model.fit()

beta = results.params['SPY_returns_percent']
print(f"OLS beta: {beta}")
print("On average: When SPY (representative of the market) moves by 1%, NVDA moves by 2%. This means NVDA is a risky stock.")

OLS beta: 2.041560237024373
On average: When SPY (representative of the market) moves by 1%, NVDA moves by 2%. This means NVDA is a risky stock.
